# 0530-0604 imdb review

image가 아닌 text

### Import modules

embedding, global average pooling 1d, pad_sequences

In [2]:
# TensorFlow and tf.keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

### load data 
- #### dataset (x: imdb article -> y: text type)
y: 0 bad 1 good
- #### word2idx dict (key: word; value: index)

num words=vocab_zise = 10000: input node의 수를 직접 설정 (10000개만 index를 매긴다. 나머지는 unknown)
word2idx: 사전 = 단어와 그 단어의 번호를 정의

In [3]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.imdb.load_data(num_words=vocab_size)
word2idx = tensorflow.keras.datasets.imdb.get_word_index()

In [4]:
#번호(단어)의 그룹
x_train[0]
# 0 or 1 = bad or good
y_train[0]

1

### modifiy word2idx dict (to reflect x_train, x_text)

modify하는 이유

In [5]:
#word2idx.items()= 각 단어 dict- 앞에 있는게 key, 뒤에 있는게 value
len(word2idx)

88584

word2idx = {k:v+3 for k, v in word2idx.items()}
in word2idx.items()- 88588번 돌면서 dict를 만듬 그냥 k:v 했으면 똑같은 dict가 나올 것
단 k:v+3 ->원래의 dict의 value값에 3만 더해주는 것 ->PAD, START, UNK, UNUSED를 넣기 위해서
이 네줄을 넣어서 사전은 update했다

In [6]:
word2idx = {k:v+3 for k, v in word2idx.items()}
word2idx['<PAD>'] = 0
word2idx['<START>'] = 1
word2idx['<UNK>'] = 2
word2idx['<UNUSED>'] = 3

### create idx2word: inverse of word2idx

위에서 사전update를 하고 inverse version을 또 만들어야 함
숫자값을 알면 그 단어가 뭐였는지 재참조를 하는 것

In [7]:
idx2word = {v:k for k, v in word2idx.items()}

### padding x_train, x_test

maxlen=256: 최대 256개까지
padding='post': 뒷부분에 padding을 해줘라
 value=word2idx['<PAD>'] 앞에서 정해준 value값으로 padding

In [8]:
#아래 작업 전에는 len = 218
len(x_train[0])

218

In [9]:
x_train = pad_sequences(x_train, value=word2idx['<PAD>'], padding='post', maxlen=256)
x_test = pad_sequences(x_test, value=word2idx['<PAD>'], padding='post', maxlen=256)

In [10]:
#위 작업 후에는 pad로 채워져 256 (content를 보면 그 value값인 0으로 채워짐)
len(x_train[0])

256

In [11]:
x_train[0]
#이 256개의 숫자를 256x10000 으로 바꿔주는 것이 밑의 embedding(차원을 1차원에서 2차원으로 바꿈)

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

어떤 review comment던지 256x10000의 matrix
128의 hidden layer을 거치고?


### create model

embedding의 input: 256개의 숫자
embedding이 하는 일은 256x10000을 하고 128개의 hidden layer까지 만들어줌
그 다움 gap로 average내줌

# 0604

##### embedding의 의미
    - model.add(Embedding(vocab_size, 128))에서 추가
    - model.add(Embedding(vocab_size, 128, 256)) ->256은 input_length
    
    - ex)756이라는 숫자가 들어온다고 할 때
    1st layer) 10000개의 node(vocab_size)에서 756째 node는 1이고 나머지는 0
    2nd layer) 128개 node
    layer을 잇는 weight가 필요 (*756을 입력 할 때는 weight 필요 없음 onyl 입력값 일 뿐)
    = 756 - 10000 - 128 까지가 embedding
    -> 256 x1 / 256x10000 / 10000x128 / 256x128 ->여기에서 마지막의 256이 들어가는 부분이 embedding의 세번째 부분(input_length = 단어의 길이)


In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.summary()

In [ ]:
Adam = tensorflow.keras.optimizers.Adam
model.compile(optimizer = Adam(lr=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.input_shape)
print(model.output_shape)

### train model

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_split=0.33)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### test model

In [ ]:
model.evaluate(x_test, y_test)

### inference

256개의 단어- reshape! 이보다 적을때는 pad

In [ ]:
testID = 100
print(' '.join([idx2word[idx] for idx in x_test[testID]]))

out = model.predict(x_test[testID].reshape(1,256))
print(y_test[testID])
print(out)

In [ ]:
len(x_test[100])